# Requirement 4
Competition amongs multiples regret minimizer in the auction problem context.

In [ ]:
import numpy as np
#import jax.numpy as np
import matplotlib.pyplot as plt
import Configuration as config
from scipy import stats
from scipy import optimize
from tqdm import tqdm
from functools import wraps
import time
from scipy.optimize import curve_fit

### UCB-like algorithm

In [ ]:
class UCB_agent:
    def __init__(self, B, T, auctions, prices, n_advertisers, n_users, my_valuation, eta=0.1) -> None:
        self.B = B          # budget
        self.T = T          # number of rounds
        self.auctions = auctions
        self.arms = prices  # arms of the UCB algorithm -> discretization of the prices
        self.eta = eta      # learning rate
        self.N_pulls = np.zeros(len(self.arms))     # number of pulls for each arm
        self.t = 0          # time step
        self.a_t = None     # arm played at time t
        self.n_advertisers = n_advertisers          # number of advertisers (excluded the agent)
        self.rho = B/T      # budget per round
        self.n_users = n_users
        self.my_valuation = my_valuation         # CHOSEN RANDOMLY, CHECK
        self.budget_depleted = False
        self.average_rewards = np.zeros(len(self.arms))    # average rewards for each arm
        
    def action(self):
        # choose price to bid
        if self.B <= 1:
            if not self.budget_depleted:
                print('Budget depleted at round',self.t)
                self.budget_depleted = True
            self.a_t = 0        # cannot bid because the budget is depleted
        elif self.t < len(self.arms):     # have not pulled all arms yet
            self.a_t = self.arms[self.t]
        else:
            # arm choosen by UCB algorithm
            ucbs = self.average_rewards + np.sqrt(2*np.log(self.T*self.auctions)/self.N_pulls)
            self.a_t = self.arms[np.argmax(ucbs)]
        return self.a_t

    def update(self, f,c):
        arm_played_index = np.where(self.arms==self.a_t)[0][0]      # bad code to get the index of an element, no index() bc it is a numpy array
        self.N_pulls[arm_played_index] += 1    # we update the number of pulls for the arm played
        self.average_rewards[arm_played_index] += (f - self.average_rewards[arm_played_index])/self.N_pulls[arm_played_index]
        self.t += 1     # incresing time step
        self.B -= c     # updating the budget

### Primal-dual algorithm for truthful auctions 

In [ ]:
class primal_dual_agent:
    def __init__(self, B, T, auctions, prices, n_advertisers, n_users, my_valuation, eta=0.1):
        self.B = B
        self.T = T
        self.auctions = auctions
        self.eta = eta
        self.rho = B/(T*auctions)
        self.v = my_valuation
        self.arms = prices
        self.a_t = None
        self.lambda_t = 0     # dual variable
        self.budget_depleted = False
    def action(self):
        # choose price to bid
        if self.B <= 1:
            if not self.budget_depleted:
                #print('Budget depleted at round',self.t)
                self.budget_depleted = True
            self.a_t = 0        # cannot bid because the budget is depleted
        else:
            self.a_t = self.v/(1+self.lambda_t)

        return self.a_t

    def update(self,f,c):
        self.lambda_t = max(0, min(self.lambda_t - self.eta*(self.rho - c), 1/self.rho))     # projection onto [0,1/rho]
        #self.t += 1     # incresing time step
        self.B -= c     # updating the budget

### Primal-dual algorithm for non-truthful auctions 

In [ ]:
class HedgeAgent:
    def __init__(self, narms, learning_rate): #ho tolto il numero di slot da questa classe per tenere solo narms
        self.narms = narms   #number of possible bids
        self.learning_rate = learning_rate #supposing it's the rate at which the algorithm progress
        self.weights = np.ones(narms) #weight == proportionnal to the probabilty
        self.x_t = np.ones(narms)/narms #probability of a bid to be played
        self.a_t = None # bid choice done during the round
        self.t = 0

    def pull_arm(self): #choose which arm to play
        self.x_t = self.weights/sum(self.weights)
        self.a_t = np.random.choice(np.arange(self.narms), p=self.x_t)
        return self.a_t

    def update(self, l_t): #update the weights
        self.weights *= np.exp(-self.learning_rate*l_t)
        self.t += 1

In [ ]:
class FFMultiplicativePacingAgent:
    def __init__(self, bids_set, valuation, budget, T, eta, lambdas):
        self.bids_set = bids_set #available choices
        self.K = len(bids_set) # number of possible bids
        self.hedge = HedgeAgent(self.K, np.sqrt(np.log(self.K)/T))
        self.valuation = valuation #what you earn, ctr*profit_per_click
        self.budget = budget
        self.eta = eta #learning rate for the pacing, i.e. how much we update the pacing multiplier
        self.T = T  #number of rounds
        self.rho = self.budget/self.T #bid limit
        self.lmbd = 1 #pacing multiplier, when we spend more than rho it increses otherwise we decrease it
        self.t = 0
        self.lambdas = lambdas

    def bid(self):
        if self.budget < 1:
            return 0
        return self.bids_set[self.hedge.pull_arm()]

    def update(self, f_t, c_t, sorted_bids):
        # update hedge
        f_t_full = np.zeros(self.K)
        c_t_full = np.zeros(self.K)
        for i, b in enumerate(self.bids_set): # full (expert) feedback
            if (get_slot_won(b, sorted_bids, len(self.lambdas)) is not None):
                f_t_full[i] = self.valuation*self.lambdas[get_slot_won(b, sorted_bids, len(self.lambdas))] - b
                c_t_full[i] = b
                
        L = f_t_full - self.lmbd*(c_t_full-self.rho)
        range_L = 2+(1-self.rho)/self.rho
        self.hedge.update((2-L)/range_L) # hedge needs losses in [0,1]
        # update lagrangian multiplier
        self.lmbd = np.clip(self.lmbd-self.eta*(self.rho-c_t),
                            a_min=0, a_max=1/self.rho)
        # update budget
        self.budget -= c_t

### Auctions classes

In [ ]:
class Auction:
    def __init__(self, *args, **kwargs):
        pass

    def get_winners(self, bids):
        pass

    def get_payments_per_click(self, winners, values, bids):
        pass

    def round(self, bids):
        winners, values = self.get_winners(bids) # allocation mechanism!
        payments_per_click = self.get_payments_per_click(winners, values, bids)
        return winners, payments_per_click


In [ ]:
class FirstPriceAuction(Auction):
    def __init__(self, ctrs, lambdas):
        self.ctrs = ctrs
        self.lambdas = lambdas
        self.n_adv = len(self.ctrs)
        self.n_slots = len(self.lambdas)
    
    def get_winners(self, bids):
        adv_values = self.ctrs*bids
        adv_ranking = np.argsort(adv_values) 
        winners = np.flip(adv_ranking[-self.n_slots:]) # important to flip the order from higher to lower
        winners_values = adv_values[winners]
        return winners, winners_values
    
    def get_payments_per_click(self, winners, values, bids):
        payment = bids[winners]
        return payment.round(2)

# Simulation

In [ ]:
def run_simulation(n_trials, auctions, iterations, n_users, cost, conversion_probability, n_arms, budget, lambdas, n_advertisers, prices, my_valuation, pricing_env, bidding_env, pricing_agent, bidding_agent, best_price):
  # empty arrays for later logging
  utilities = np.empty((n_trials, iterations*auctions))
  my_bids = np.empty((n_trials, iterations*auctions))
  my_payments = np.empty((n_trials, iterations*auctions))
  my_prices = np.empty((n_trials, iterations))                # chosen one price for each iteration, not for each auction
  total_wins_per_trial = np.zeros(n_trials)
  cumulative_regret_payments = np.array([])
  pricing_regret_per_trial = list()
  tmp_pricing_rewards = list()
  m_t_history = list()
  bidding_regret_per_trial = list()


  for trial in tqdm(range(n_trials)):
    m_t_history = []
    # creating new environments and agents for each independent trial
    pa = pricing_agent(iterations, n_users, cost, conversion_probability, n_arms )
    ba = bidding_agent(budget, iterations, auctions, prices, n_advertisers,n_users, my_valuation)

    pse = pricing_env(conversion_probability, cost)
    bse = bidding_env(B=budget, T=iterations, lambdas=lambdas, n_advertisers=n_advertisers)

    for round in range(iterations):
        p_t = pa.pull_arm()    # pricing agent choose a price p for the product
        my_prices[trial, round] = p_t
        for auction in range(auctions):
            my_bid = ba.action()    # bidding agent decides how much to bid
            other_bids = np.random.uniform(0, 1, size = (n_advertisers)) # size = (N_ADVERTISERS, N_USERS)
            all_bids = np.append(my_bid, other_bids)    # all bids, including mine in first position
            winners, payments_per_click = bse.round(bids=all_bids)    # somebody wins the auction
            my_win = 0 if 0 not in winners else 1       # boolean value which tells if my ad is shown in a slot

            if my_win:
                my_index = np.where(winners==0)[0][0]    # index of the slot where my ad is shown
                #print(payments_per_click[my_index])
                f_t, c_t = (my_valuation-payments_per_click[my_index])*my_win, payments_per_click[my_index]*my_win      # could remove my_win
            else:
                f_t, c_t = 0, 0
            ba.update(f_t, c_t)

            # logging
            utilities[trial, round*auctions+auction] = f_t
            my_bids[trial, round*auctions+auction] = my_bid
            my_payments[trial, round*auctions+auction] = c_t
            total_wins_per_trial[trial] += my_win
            m_t_history.append(c_t)

        d_t, r_t = pse.round(p_t, n_t=n_users)
        pa.update(r_t/n_users)


    tmp_pricing_rewards.append(pa.get_rewards())
    cumulative_regret_payments = np.cumsum(expected_clairvoyant_rewards-pa.get_rewards())
    pricing_regret_per_trial.append(cumulative_regret_payments)
    ## I compute my sequence of utilities at every round
    utilities_of_all_auctions = [(my_valuation-m_t_history[i])*(my_valuation>=m_t_history[i]) for i in range(len(m_t_history))]
    sorted_round_utility = np.flip(np.argsort(utilities_of_all_auctions)) # sorted rounds, from most profitable to less profitable
    clairvoyant_utilities = np.zeros(iterations*auctions)
    clairvoyant_bids= np.zeros(iterations*auctions)
    clairvoyant_payments = np.zeros(iterations*auctions)
    c = 0
    i = 0
    while c <= budget-1 and i < iterations*auctions:
        clairvoyant_bids[sorted_round_utility[i]] = 1
        clairvoyant_utilities[sorted_round_utility[i]] = utilities_of_all_auctions[sorted_round_utility[i]]
        clairvoyant_payments[sorted_round_utility[i]] = m_t_history[sorted_round_utility[i]]
        c += m_t_history[sorted_round_utility[i]]
        i+=1
    bidding_regret_per_trial.append(np.cumsum(clairvoyant_utilities - utilities[trial]))

  return my_payments, my_bids, utilities, my_prices, total_wins_per_trial, pricing_regret_per_trial, tmp_pricing_rewards, bidding_regret_per_trial
